In [41]:
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import plotly.express as px

#these are feiner naming conventions

# datafr = datafr.rename(columns={"Time": 'Time Stamp',
# 'pCO2 (mmhg)': 'pCO2_mmHg_',
# 'pO2 (mmHg)':'pO2_mmHg_',
# 'sO2 (%)': 'sO2_%_',
# 'COHb (%)':'COHB_%_',
# 'MetHb (%)':'MetHb_%_',
# 'tHb (g/dL)':'tHb_g_dL_',
# 'Accession number':'UPI',})


 qulaity checks
 - there is a missing accession number
 - ? there are duplicate 'patient IDs' between the two files

issues to deal with
- how many ABG files? 2? 3? maybe just one?

ingest multiple CSVs

delete columns not needed

 input/select the session for each patient
 maybe query the 
- 1 use the UPI to query the patient redcap database for the patient name, and 
- 2 use the UPI to query the session database for the associated sessions and dates

for each upi, create a dropdown selectbox of sessions and dates


In [42]:
df1 = pd.read_csv('c1.csv', encoding='ANSI')
df2 = pd.read_csv('c2.csv', encoding='ANSI')

In [43]:
df_join = pd.concat([df1,df2])

In [44]:
def feinerize(datafr):
    #separate timestamp into two columns
    datafr['Time'] = pd.to_datetime(datafr['Time'])
    datafr['Date Calc'] = datafr['Time'].dt.date
    datafr['Time Calc'] =  datafr['Time'].dt.time

    #separate patient ID into two columns
    datafr[['Subject', 'Sample']] = datafr['Patient ID'].astype(str).str.split(pat='.', expand=True)

    # rename columns 
    datafr = datafr.rename(columns={"Time": 'Time Stamp',
    'pCO2 (mmHg)': 'pCO2',
    'pO2 (mmHg)':'pO2',
    'sO2 (%)': 'sO2',
    'COHb (%)':'COHb',
    'MetHb (%)':'MetHb',
    'tHb (g/dL)':'tHb',
    'Accession number':'UPI',
    'K+ (mmol/L)':'K+',
    'Na+ (mmol/L)':'Na+',
    'Ca++ (mmol/L)':'Ca++',
    'Cl- (mmol/L)':'Cl-',
    'Glu (mmol/L)': 'Glucose',
    'Lac (mmol/L)':'Lactate',
    'p50(act) (mmHg)':'p50',
    'cBase(Ecf) (mmol/L)':'cBase'})

    return datafr[allcols]

In [45]:
#fcols = feiner columns that he uses for processing
fcols = ['Time Stamp', 'Date Calc', 'Time Calc', 'Subject', 'Sample', 'Patient ID', 'pH','pCO2', 'pO2', 'sO2','COHb','MetHb','tHb','UPI']

# allcols = the rest of the columns we actually want + feiner cols
allcols = fcols + ['K+', 'Na+','Ca++','Cl-','Glucose','Lactate','p50','cBase']

In [46]:
df_join = feinerize(df_join)

In [47]:
df_join

,Time Stamp,Date Calc,Time Calc,Subject,Sample,Patient ID,pH,pCO2,pO2,sO2,...,tHb,UPI,K+,Na+,Ca++,Cl-,Glucose,Lactate,p50,cBase
0,2023-08-09 14:59:00,2023-08-09,14:59:00,5,25,5.25,7.425,40.9,37.9,70.6,...,11.9,NaN,3.6,140,1.23,103,5.3,0.5,27.50,2.4
1,2023-08-09 14:58:00,2023-08-09,14:58:00,5,23,5.23,7.425,41.1,37.0,68.5,...,11.9,1136.0,3.7,140,1.22,103,5.1,0.4,27.78,2.6
2,2023-08-09 14:55:00,2023-08-09,14:55:00,5,21,5.21,7.433,39.7,45.1,80.9,...,11.8,NaN,3.6,140,1.23,104,5.1,0.4,26.74,2.2
3,2023-08-09 14:54:00,2023-08-09,14:54:00,5,19,5.19,7.429,40.1,45.8,81.0,...,11.7,1136.0,3.6,140,1.23,104,5.1,0.4,27.12,2.2
4,2023-08-09 14:51:00,2023-08-09,14:51:00,5,17,5.17,7.424,40.3,59.9,90.8,...,11.7,1136.0,3.6,140,1.23,104,5.1,0.4,26.35,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2023-08-09 09:47:00,2023-08-09,09:47:00,1,1,1.10,7.436,42.5,39.9,74.8,...,11.7,1096.0,4.3,140,1.20,103,4.9,1.0,26.74,4.4
58,2023-08-09 09:44:00,2023-08-09,09:44:00,1,8,1.80,7.433,41.6,46.0,81.6,...,11.5,1096.0,4.1,141,1.17,105,4.6,1.0,26.78,3.5
59,2023-08-09 09:43:00,2023-08-09,09:43:00,1,6,1.60,7.438,42.8,47.3,83.4,...,11.6,1096.0,4.6,140,1.22,103,4.7,1.1,26.30,4.8
60,2023-08-09 09:40:00,2023-08-09,09:40:00,1,4,1.40,7.413,42.8,63.6,92.5,...,11.3,1096.0,4.1,141,1.21,105,4.4,1.0,25.79,2.8
